In [23]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 

In [24]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL)

#### Step1: 
    Use BeautifulSoup to work on webpage

In [25]:
soup = BeautifulSoup(r.content, 'html5lib') 
#print(soup.prettify())

#### Step2: 
    Extract only the table that is needed

In [26]:
table = soup.find('table', attrs = {'class':'wikitable sortable'}) 
#print(table.prettify())  


#### Step3: 
    Fetch the header row to create the dataframe

In [5]:
cols=[]
for row in table.findAll('tr'):
    for headers in row.findAll('th'):
        cols.append(headers.text.rstrip("\n\r"))
        
cols

['Postcode', 'Borough', 'Neighbourhood']

#### Step4: 
    Create a dataframe with the data from table

In [6]:
df = pd.DataFrame(columns=cols)
row_data = []

for row in table.findAll('tr'):
    for data in row.findAll('td'):
        row_data.append(data.text.rstrip("\n\r"))
    if len(row_data)==3:
        df=df.append(pd.DataFrame([row_data], columns=cols), ignore_index = True)
    row_data = []

df.head(11)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [7]:
df.shape

(288, 3)

#### Step5: 
    Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [8]:
df = df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [9]:
df.shape

(211, 3)

#### Step6:
 More than one neighborhood can exist in one postal code area.  For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park.   These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [10]:
df1=df.groupby(['Postcode'], as_index=False, sort=False)['Borough'].first()
df2=df.groupby(['Postcode'], as_index=False, sort=False)['Neighbourhood'].agg(lambda x: ', '.join(x.values))
df=pd.merge(df1, df2, on=["Postcode"], how="inner")
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [11]:
df.shape

(103, 3)

#### Step7: 
    If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [12]:
df['Neighbourhood'] = df.apply(lambda x: x['Borough'] if x['Neighbourhood'] == 'Not assigned' else x['Neighbourhood'], axis=1)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [13]:
df.shape

(103, 3)

#### Step8: 
    Import and get coordinates from geocoder

In [14]:
#!conda install -c conda-forge geocoder --yes

In [15]:
import geocoder 

In [16]:
# Class definition to generate latitude and logitude for a given postal_code
def geo_loc(postal_code):
        # initialize your variable to None
    lat_lng_coords = None
    i=0
         # loop until you get the coordinates
    print(postal_code)
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        print("Queried Geocode :", i)
        i += 1
    return(lat_lng_coords)

In [17]:
df['Latitude'] = df['Postcode'].apply(lambda x: (geo_loc(x))[0])
df['Longitude'] = df['Postcode'].apply(lambda x: (geo_loc(x))[1])

M3A
Queried Geocode : 0
Queried Geocode : 1
Queried Geocode : 2
Queried Geocode : 3
Queried Geocode : 4
Queried Geocode : 5
Queried Geocode : 6
Queried Geocode : 7
Queried Geocode : 8
Queried Geocode : 9
Queried Geocode : 10
Queried Geocode : 11
Queried Geocode : 12
Queried Geocode : 13
Queried Geocode : 14
Queried Geocode : 15
Queried Geocode : 16
Queried Geocode : 17
Queried Geocode : 18
Queried Geocode : 19
Queried Geocode : 20
Queried Geocode : 21
Queried Geocode : 22
Queried Geocode : 23
Queried Geocode : 24
Queried Geocode : 25
Queried Geocode : 26
Queried Geocode : 27
Queried Geocode : 28
Queried Geocode : 29
Queried Geocode : 30
Queried Geocode : 31
Queried Geocode : 32
Queried Geocode : 33
Queried Geocode : 34
Queried Geocode : 35
Queried Geocode : 36
Queried Geocode : 37
Queried Geocode : 38
Queried Geocode : 39
Queried Geocode : 40
Queried Geocode : 41
Queried Geocode : 42
Queried Geocode : 43
Queried Geocode : 44
Queried Geocode : 45


KeyboardInterrupt: 

#### Step9:
    Get the coordinates from csv as the geocoder package is not responding

In [18]:
coor_df = pd.read_csv("http://cocl.us/Geospatial_data")

In [19]:
coor_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
coor_df.columns = ['Postcode', 'Latitude', 'Longitude']

In [21]:
df = pd.merge(df, coor_df, on=["Postcode"], how="inner")

In [22]:
df.head(11)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
